In [3]:
# Imports
import tensorflow as tf
from pathlib import Path
import numpy as np

In [4]:
# Load Data Set
(x_train, y_train), (x_test, y_test) =tf.keras.datasets.cifar10.load_data()

In [5]:
# Normalize the data in 0 - 1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255

In [6]:
# Convert class vectors to binary class matrices 
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [8]:
# Create a model and add layers
from tabnanny import verbose


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2))) # Good to put a MaxPooling layer after each block of Convolutional Layers
model.add(tf.keras.layers.Dropout(0.25)) # Good to add after each MaxPooling layer

model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))# Good to put a MaxPooling layer after each block of Convolutional Layers
model.add(tf.keras.layers.Dropout(0.25))# Good to add after each MaxPooling layer

model.add(tf.keras.layers.Flatten()) # Should flatten when transitioning from Convolutional Layer to Dense Layer

model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.50)) # Good to add after a group of Dense Layers
model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Add early stopping to the model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1)

# Compile the Model
model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer =  'adam',
    metrics = ['accuracy']    
)

# Print a summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_5 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 15, 15, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 15, 15, 64)        18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        36928     
                                                      

In [9]:
# Train the model
from random import shuffle


model.fit(
    x_train, 
    y_train,
    batch_size = 32,
    epochs = 30,
    validation_data = (x_test, y_test),
    shuffle = True,
    callbacks = [callback]    
)

# Save neural network sturcture
model_structure = model.to_json()
f = Path('modlel_structure.json')
f.write_text(model_structure)

# Save neural network trained weights
model.save_weights('model_weights.h5')


Epoch 1/30
1563/1563 [==============================] - 45s 28ms/step - loss: 1.5215 - accuracy: 0.4444 - val_loss: 1.1638 - val_accuracy: 0.5875
Epoch 2/30
1563/1563 [==============================] - 42s 27ms/step - loss: 1.0927 - accuracy: 0.6114 - val_loss: 0.9034 - val_accuracy: 0.6875
Epoch 3/30
1563/1563 [==============================] - 43s 27ms/step - loss: 0.9375 - accuracy: 0.6680 - val_loss: 0.8401 - val_accuracy: 0.7073
Epoch 4/30
1563/1563 [==============================] - 43s 28ms/step - loss: 0.8547 - accuracy: 0.6998 - val_loss: 0.7471 - val_accuracy: 0.7398
Epoch 5/30
1563/1563 [==============================] - 42s 27ms/step - loss: 0.7927 - accuracy: 0.7223 - val_loss: 0.7518 - val_accuracy: 0.7468
Epoch 6/30
1563/1563 [==============================] - 43s 28ms/step - loss: 0.7465 - accuracy: 0.7369 - val_loss: 0.7320 - val_accuracy: 0.7439
Epoch 7/30
1563/1563 [==============================] - 44s 28ms/step - loss: 0.7107 - accuracy: 0.7504 - val_loss: 0.7025 -

In [10]:
class_labels = [
    'Plane',
    'Car',
    'Bird',
    'Cat',
    'Deer',
    'Dog',
    'Frog',
    'Horse',
    'Boat',
    'Truck'
]

In [11]:
# Load the Neural Network structure
f = Path('modlel_structure.json')
model_structure = f.read_text()

# Recreate the Neural Network
model = tf.keras.models.model_from_json(model_structure)

# Reload the models trained weights
model.load_weights('model_weights.h5')

In [16]:
from tensorflow.keras.preprocessing import image

# Load an image to test with (resize to 32x32 pixels)
img = image.load_img('dodger.jpg', target_size=(32,32))

# Convert image to numpy array
image_to_test = image.img_to_array(img) / 255 # Scale image from 0 to 1

# Create a Batch of images as a 4D array
list_of_images = np.expand_dims(image_to_test, axis=0)

In [17]:
# Make a prediction using the model
results = model.predict(list_of_images)

# Since we only tested one image, we only need to check the first result
single_result = results[0]

# Get the likelihood of all possible classes
most_like_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_like_class_index]

# Get the name of the most likely class
class_label = class_labels[most_like_class_index]

# Print the result
print(f'This image is a {class_label} - Likelihood: {class_likelihood}')

This image is a Car - Likelihood: 0.7175264954566956
